### This scraper is for grabbing the first 1024 elements of the PROPERTY ID search with a blank string as an input.

- It is only to act as a test DB until full Database is acquired.
- Itterating over rows can be a very lengthy process so it is not for a full production environment but will serve in a pinch

In [ ]:
import os
import pandas as pd
os.getcwd()

https://smu.bootcampcontent.com/SMU-Coding-Bootcamp/smu-dal-data-pt-09-2021-u-c/-/blob/master/01-Lesson-Plans/12-Web-Scraping-and-Document-Databases/2/Activities/09-Ins_Pandas_Scraping/Solved/Ins_Pandas_Scraping.ipynb

In [ ]:
# Pandas Scrape

# url with no entry in search box
url_no_query = 'https://iswdataclient.azurewebsites.net/webSearchID.aspx?dbkey=midlandcad'

# search query was blank string, not automated.
url = 'https://iswdataclient.azurewebsites.net/webSearchID.aspx?dbkey=midlandcad&stype=id&sdata=R&time=202201100954042#top'

In [ ]:
# read_html returns list of table elements
CAD_results_list = pd.read_html(url)
CAD_results_list

-----------------------------

### Notes:

- we have domain table
- get results for individual properties, pull values and drop to list | dictionary as needed
- create secondary df from list of information from 'view property'link
- merge to domain table on PROPERTY_ID
- clean
- save to_json('filepath.json')


In [ ]:
# set variable_df == list element corresponding to search results
CAD_results = CAD_results_list[13]
# extract columns to drop unnecessary colums
CAD_df_columns = CAD_results.columns.to_list()
CAD_df_columns

In [ ]:
#rename columns
CAD_results = CAD_results.rename(columns={
    'Details': 'details',
    'Property ID': 'property_id',
    'Geographic ID': 'geographic_id',
    'Owner Name': 'owner_name',
    'Address': 'address',
    'Legal': 'legal',
    'Market Value': 'market_value',
    'Unnamed: 7': 'unnamed_7'
})

#drop unnecessary column(s)

CAD_results = CAD_results.drop(columns=['details','unnamed_7'])
CAD_results

### Each table element has a {key: {integer_key : value_we_want}}

In [ ]:
row = CAD_results.iloc[[0]]
row_dict = row.to_dict()
row_dict

In [ ]:
CAD_results.iloc[[15]]
row_dict = row.to_dict()
row_dict


# key = row_dict['address'][0]
# str(key)

In [ ]:
# save results table to json for transfer to mongoDB
import pymongo

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare Database
db = client.CAD_search_results

# declare collection(s)

# collection for the results of the main search page, PROPERTY ID search
bulk_results = db.bulk_results

# collection for the "view_property_results" link on main search results page
# view_property_results = db.view_property_results


In [ ]:
# moving index values to variable in event of repeating this search technique with more searches
index = CAD_results.index.to_list()
index_length = len(index)

In [ ]:
for i in index:
    # row = row of index value, i
    row = CAD_results.iloc[[i]]
    # turns row of df into dictionary to allow to inserting to MongoDB
    row_dict = row.to_dict()
    clean_row = {}
    clean_entry = {}
    # edit the row to equal only {key:value} NOT {key:{key_2: value}}
    clean_row['property_id']= row_dict['property_id'][i]
    clean_row['geographic_id']= row_dict['geographic_id'][i]
    clean_row['owner_name']= row_dict['owner_name'][i]
    clean_row['address']= row_dict['address'][i]
    clean_row['legal']= row_dict['legal'][i]
    clean_row['market_value']= row_dict['market_value'][i]
    key = row_dict['address'][i]
    if key == 'nan':
        print('key is nan')
    
    
#     try:

#         clean_entry[key]= clean_row

#         bulk_results.insert_one(clean_entry)
#         print()
#         print(f'Row {i} saved to "bulk_results" in MongoDB')
#         print('-----------------------------------')
        
#     except InvalidDocument as error:
#         print('skipping')



## Verify data integrity by importing documents as DataFrame.

In [ ]:
doc_list = []

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare Database
db = client.CAD_search_results

# declare collection(s)

# collection for the results of the main search page, PROPERTY ID search
bulk_results = db.bulk_results

# collection for the "view_property_results" link on main search results page
view_property_results = db.view_property_results

# declare the collections

collection = bulk_results

# pull al documents from collection specified above
documents = collection.find()
for doc in documents:
    # move each doc as dictionary to list to create a list of dictionaries
    doc_list.append(doc)

In [ ]:
# read list of dictionaries (doc_list) as DataFrame

bulk_results_df = pd.DataFrame(doc_list)

# drop _id column, keep everything else.
bulk_results_df = bulk_results_df.drop(columns=['_id'])

# display results to verify integrity of data.
bulk_results_df

In [ ]:
# save results table to json format to create a back up of database collection.
bulk_results_df.to_json('..\\raw_data\\test_dataframe.json')
